In [42]:
from pulp import *
import pandas as pd

In [43]:
#Read data in to data frame (Only first 64 rows)
df = pd.read_excel("diet_large.xls", nrows=7146, skiprows=[0])

df.fillna(0, inplace=True) #fill nan with 0

#Check data 
df.head()
df.tail()

,Long_Desc,Protein,"Carbohydrate, by difference",Energy,Water,Energy.1,"Calcium, Ca","Iron, Fe","Magnesium, Mg","Phosphorus, P",...,Riboflavin,Niacin,Pantothenic acid,Vitamin B-6,"Folate, total",Vitamin B-12,Vitamin K (phylloquinone),Cholesterol,"Fatty acids, total trans","Fatty acids, total saturated"
7141,"Frog legs, raw",16.4,0.0,73,81.9,305,18.0,1.5,20.0,147.0,...,0.25,1.2,0.0,0.12,15.0,0.4,0.1,50.0,0.0,0.076
7142,"Fish, mackerel, salted",18.5,0.0,305,43.0,1276,66.0,1.4,60.0,254.0,...,0.19,3.3,0.0,0.41,15.0,12.0,7.8,95.0,0.0,7.148
7143,"Mollusks, scallop, (bay and sea), cooked, steamed",23.2,0.0,112,73.1,469,115.0,3.0,55.0,338.0,...,0.06,1.3,0.0,0.10,12.0,1.3,0.2,53.0,0.0,0.146
7144,"Mollusks, snail, raw",16.1,2.0,90,79.2,377,10.0,3.5,250.0,272.0,...,0.12,1.4,0.0,0.13,6.0,0.5,0.1,50.0,0.0,0.361
7145,"Turtle, green, raw",19.8,0.0,89,78.5,372,118.0,1.4,20.0,180.0,...,0.15,1.1,0.0,0.12,15.0,1.0,0.1,50.0,0.0,0.127


In [44]:
#Create a list of all foods
food_list = list(df['Long_Desc'])


#Create a dictionary of price (all foods)
cholesterol = dict(zip(food_list , df['Cholesterol']))


#Create a list of nutrition facts names and min and max of each one
col_name = list(df.columns[1:])


val_min = [     56,     130,    2400,    3700,    2400, 1000,  8, 270,  700,    4700, 1500, 11, 0.9, 2.3,  55,  900,   15,  200,   90,  0.0012,     1.3, 16,       5, 1.3,  400,     2.4,     120]

val_max = [1000000, 1000000, 1000000, 1000000, 1000000, 2500, 45, 400, 4000, 1000000, 2300, 40,  10,  11, 400, 3000, 1000, 2000, 2000, 1000000, 1000000, 35, 1000000, 100, 1000, 1000000, 1000000]



#Create a list of nutrition_facts list that containes a dictionary of foods and it's nutrition(calories, cholesterol, ...)
nutrition_facts = []
for i,j in enumerate(col_name):
    nutrition_facts.append(dict(zip(food_list , df[col_name[i]])))

In [45]:
#Define variables
food_vars = LpVariable.dicts("Food", food_list, lowBound=0, cat=LpContinuous)

In [46]:
#Create Objective function (Minimization)
problem = LpProblem("Food_cholesterol", LpMinimize)
problem += lpSum([cholesterol[food] * food_vars[food] for food in food_list])

In [47]:
#Add Constraints
for i,n_name in enumerate(col_name):
    if i < len(val_min):
        problem += lpSum(nutrition_facts[i][food] * food_vars[food] for food in food_list) >= val_min[i], 'Minimum ' + n_name
        problem += lpSum(nutrition_facts[i][food] * food_vars[food] for food in food_list) <= val_max[i], 'Maximum ' + n_name
    else:
        problem += lpSum(nutrition_facts[i][food] * food_vars[food] for food in food_list) >= 0, 'Minimum ' + n_name

In [48]:
#Solve the optimization problem
problem.solve()

1

In [49]:
for var in problem.variables():
    if (var.varValue > 0):
        print(var.name, "==>", var.varValue)

Food_Beans,_adzuki,_mature_seeds,_raw ==> 0.059863415
Food_Broccoli_raab,_raw ==> 0.069514608
Food_Cocoa_mix,_no_sugar_added,_powder ==> 0.42866218
Food_Egg,_white,_dried,_flakes,_glucose_reduced ==> 0.14694398
Food_Infant_formula,_MEAD_JOHNSON,_ENFAMIL,_NUTRAMIGEN,_with_iron,_p ==> 0.73805891
Food_Infant_formula,_NESTLE,_GOOD_START_ESSENTIALS__SOY,__with_iron, ==> 0.4258564
Food_Infant_formula,_ROSS,_ISOMIL,_with_iron,_powder,_not_reconstitu ==> 0.050114149
Food_Margarine_like_spread,_approximately_60%_fat,_tub,_soybean_(hyd ==> 0.15033656
Food_Mung_beans,_mature_seeds,_raw ==> 0.25918767
Food_Nuts,_mixed_nuts,_dry_roasted,_with_peanuts,_with_salt_added ==> 0.18052856
Food_Oil,_vegetable,_sunflower,_linoleic,_(hydrogenated) ==> 1.184482
Food_Seeds,_sunflower_seed_kernels,_dry_roasted,_with_salt_added ==> 0.10375187
Food_Snacks,_potato_chips,_fat_free,_made_with_olestra ==> 0.031866196
Food_Spices,_paprika ==> 0.070710308
Food_Tomatoes,_sun_dried ==> 0.55106575
Food_Water,_bottled,_non

In [50]:
print(value(problem.objective))

0.0
